# Fact and Dimension Tables with Star Schema

### Import library

In [1]:
import psycopg2

In [2]:
try: 
    conn = psycopg2.connect("dbname=music user=postgres password=admin")
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)

### Get cursor & set autocommit to true

In [4]:
try: 
    cur =conn.cursor() 
except psycopg2.Error as e: 
    print("Error: Could not make cursor")
    print(e)
    
conn.set_session(autocommit = True)


### Create Fact table & Insert data

In [9]:
try:
    cur.execute("CREATE TABLE IF NOT EXISTS customer_transcations(customer_id integer, store_id integer, spent numeric)")
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)

## Insert data into the table:-

try:
    cur.execute("INSERT INTO customer_transcations(customer_id,store_id,spent)\
                VALUES(%s,%s,%s)",(1, 1, 20.50))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)
    
try: 
    cur.execute('INSERT INTO customer_transcations (customer_id, store_id, spent) \
                VALUES (%s, %s, %s)', \
               (2, 1, 35.21))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)


### Create dimension table

In [17]:
## Dimension table customer:-

try:
    cur.execute("CREATE TABLE IF NOT EXISTS customer(customer_id integer, name varchar, rewards boolean)")
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)


## Dimension table store:-

try:
    cur.execute("CREATE TABLE IF NOT EXISTS purchased_items (customer_id integer, item_number integer, item_name varchar)")
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)

## Dimension table items_purchased:-

try:
    cur.execute("CREATE TABLE IF NOT EXISTS store(store_id integer, item_name varchar)")
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)

### Insert data into the dimension tables

In [13]:
try: 
    cur.execute('INSERT INTO customer (customer_id, name, rewards) \
                VALUES (%s, %s, %s)', \
               (1, 'Amanda', True))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)
    
try: 
    cur.execute('INSERT INTO customer (customer_id, name, rewards) \
                VALUES (%s, %s, %s)', \
               (2, 'Toby', False))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)
    

In [18]:
try: 
    cur.execute('INSERT INTO purchased_items (customer_id, item_number, item_name) \
                VALUES (%s, %s, %s)', \
               (1, 1, 'Rubber Soul'))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)
try: 
    cur.execute('INSERT INTO purchased_items (customer_id, item_number, item_name) \
                VALUES (%s, %s, %s)', \
               (2, 3, 'Let it be'))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

In [16]:
try: 
    cur.execute('INSERT INTO store (store_id, state) \
                VALUES (%s, %s)', \
               (1, 'CA'))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

try: 
    cur.execute('INSERT INTO store (store_id, state) \
                VALUES (%s, %s)', \
               (2, 'WA'))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

### Query 1

In [29]:
try: 
    cur.execute("SELECT \
                    customer.name AS name, \
                    store.store_id AS store_id, \
                    store.state AS store_state, \
                    purchased_items.item_name AS purchased_item_name, \
                    customer.rewards AS rewards \
                FROM \
                    customer_transcations \
                    JOIN purchased_items ON (purchased_items.customer_id = customer_transcations.customer_id) \
                    JOIN customer ON (customer.customer_id = customer_transcations.customer_id) \
                    JOIN store ON (store.store_id = customer_transcations.store_id) \
                WHERE \
                    customer_transcations.spent > 30 \
                ")
    
    
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

('Toby', 1, 'CA', 'Let it be', False)
('Toby', 1, 'CA', 'Let it be', False)
('Toby', 1, 'CA', 'Let it be', False)


In [33]:
try:
    cur.execute("SELECT customer_id, SUM(spent) FROM customer_transcations WHERE customer_id = 1 GROUP BY customer_id")

except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

(1, Decimal('61.5'))
